In [1]:
# r"G:\2026\HawtC2\publish\Release\net9.0\HawtC2.dll"
#r "nuget:ScottPlot, 5.*"
//# r"C:\Users\123\.nuget\packages\skiasharp\2.88.8\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll"

using OpenWECD.IO.Log;
using MathNet.Numerics.LinearAlgebra;
using OpenWECD.IO.math;
using OpenWECD.IO.Numerics;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using static OpenWECD.IO.math.LinearAlgebraHelper;
using static System.Math;
using OpenWECD.FEML;
using ScottPlot;
using OpenWECD.IO.IO;

// Setup a custom formatter to display plots as images
using Microsoft.DotNet.Interactive.Formatting;
Formatter.Register(typeof(ScottPlot.Plot), (p, w) => 
    w.Write(((ScottPlot.Plot)p).GetPngHtml(800, 400)), HtmlFormatter.MimeType);

Installed Packages ScottPlot, 5.0.54

Loading extensions from `C:\Users\123\.nuget\packages\skiasharp\2.88.9\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

## 1. 模块输出的代码生成器(持续维护更新)

In [2]:
//# 设置OutputParList.xlsx路径并打开
var excle=new OpenWECD.IO.IO.MSExcle(@"G:\2026\HawtC2\OutputParList_2.0.005.xlsx");

### 1.1 创建必要函数

In [3]:

public static void writesummary(OpenWECD.IO.IO.OutFile file, string text)
{
    file.WriteLine("        /// <summary>");
    file.WriteLine($"        /// {text}");
    file.WriteLine("        /// </summary>");
}
//# 向文件当中写入title
public static string[] title=new string[]
{
"//**********************************************************************************************************************************",
"// LICENSING",
"// Copyright(C) 2021, 2025  TG Team,Key Laboratory of Jiangsu province High-Tech design of wind turbine,WTG,WL,ZZZ",
"//  ",
"//    This file is part of OpenWECD.HawtC.APIL by 赵子祯, 2021, 2025",
"// ",
"// Licensed under the Boost Software License - Version 1.0 - August 17th, 2003",
"// you may not use this file except in compliance with the License.",
"// You may obtain a copy of the License at",
"// ",
"//     http://www.HawtC.cn/licenses.txt",
"// ",
"// THE SOFTWARE IS PROVIDED \"AS IS\", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR",
"// IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,",
"// FITNESS FOR A PARTICULAR PURPOSE, TITLE AND NON-INFRINGEMENT. IN NO EVENT",
"// SHALL THE COPYRIGHT HOLDERS OR ANYONE DISTRIBUTING THE SOFTWARE BE LIABLE",
"// FOR ANY DAMAGES OR OTHER LIABILITY, WHETHER IN CONTRACT, TORT OR OTHERWISE,",
"// ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER",
"// DEALINGS IN THE SOFTWARE.",
"// 该代码由赵子祯编写的代码生成器生成,不可擅自修改!任何bug请联系赵子祯 1821216780@qq.com",
"//**********************************************************************************************************************************"
};

public static void writeinf(OpenWECD.IO.IO.OutFile file, string text="")
{
    var tilte=LinearAlgebraHelper.Copy(title);
    if(text!="")
    {
        tilte[4]=$"//    This file is part of OpenWECD.HawtC.{text} by 赵子祯, 2021, 2025";
    }
    file.WriteLine(tilte);
}
// def writeo(file,str,length):
//     lens=len(str)
//     num=length-lens
//     if(num>0):
//         for i in range(num):
// str=str+" "
//     file.write(str)
public static void writeo(OpenWECD.IO.IO.OutFile file, string str, int length)
{
    var lens=str.Length;
    var num=length-lens;
    if(num>0)
    {
        for(int i=0;i<num;i++)
        {
str=str+" ";
        }
    }
    file.Write(str,"");
}


public static (
    List<string> aname,
    List<string> name, 
    List<string> namep, 
   List<string> unit,
   List<int> channel, 
    List<string> description,
    List<string> convention,
    List<string> index,
    List<string> ipx,
    List<string> Category,
    List<string> Titlechannel,
    List<string> TitlechannelDisp
   ) getsheetdata(string sheetname,OpenWECD.IO.IO.MSExcle excle)
{
    
    //# 处理数据
    List<string> fredata = new List<string>();
    List<string> data = new List<string>();
    List<string> indetie = new List<string>();
    List<string> unit = new List<string>();
    List<int> channel = new List<int>();
    List<string> description = new List<string>();
    List<string> convention = new List<string>();
    List<string> index = new List<string>();
    List<string> ipx = new List<string>();
    List<string> Category = new List<string>();
    List<string> Titlechannel = new List<string>();
    List<string> TitlechannelDisp = new List<string>();
    //# 获取当前表格有效行数
    var sheet=excle.GetSheet(sheetname);
int row=excle.RowCount(sheet);
for (int i=1;i<row;i++)
{
    string dataTemp=sheet.RCellValue<string>(i,1).Trim();
    string dataTemp1=sheet.RCellValue<string>(i,0).Trim();
    if(dataTemp1!=string.Empty)
    {
        Category.Add(dataTemp1.Replace("/","And").Replace(" ","_").Replace("-","_").Replace("__","_"));
        Titlechannel.Add(dataTemp1=sheet.RCellValue<int>(i,7).ToString());
        TitlechannelDisp.Add(dataTemp1=sheet.RCellValue<string>(i,3));
    }
    if(dataTemp!=string.Empty)
    {
        
        data.Add(dataTemp.Trim());
        fredata.Add(dataTemp.Trim());
        indetie.Add(dataTemp.Trim());
        unit.Add(sheet.RCellValue<string>(i,5).Trim());//Units
        channel.Add( sheet.RCellValue<int>(i,7));//Convert.ToInt32(row["输出频道编号"])
        description.Add( sheet.RCellValue<string>(i,3).Trim());//"Description"
        convention.Add(sheet.RCellValue<string>(i,4).Trim());//"Convention
        string str1 = sheet.RCellValue<string>(i,8).Trim() as string ?? string.Empty;//"截面编号"
        string str2 = sheet.RCellValue<string>(i,9).Trim() as string ?? string.Empty;//"数量编号"
        string str3 = sheet.RCellValue<string>(i,10).Trim() as string ?? string.Empty;//"其他编号"
        string strres = string.Empty;
        string strres2 = string.Empty;

        if (!string.IsNullOrEmpty(str1) && string.IsNullOrEmpty(str2) && string.IsNullOrEmpty(str3))
        {
strres = $"[{str1}]";
strres2 = "[]";
        }
        else if (string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && string.IsNullOrEmpty(str3))
        {
strres = $"[{str2}]";
strres2 = "[]";
        }
        else if (string.IsNullOrEmpty(str1) && string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
        {
strres = $"[{str3}]";
strres2 = "[]";
        }
         else if (!string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && string.IsNullOrEmpty(str3))
        {
strres = $"[{str1},{str2}]";
strres2 = "[,]";
        }
        else if (!string.IsNullOrEmpty(str1) && string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
        {
strres = $"[{str1},{str3}]";
strres2 = "[,]";
        }
        else if (string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
        {
strres = $"[{str2},{str3}]";
strres2 = "[,]";
        }
        else if (!string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
        {
strres = $"[{str1},{str2},{str3}]";
strres2 = "[,,]";
        }

        index.Add(strres);
        ipx.Add(strres2);
    }

    //# 查找是否有其他名字
    string otherNames = sheet.RCellValue<string>(i,2).Trim();
    if (!string.IsNullOrEmpty(otherNames))
    {
        string[] otherNameArray = otherNames.Split(',').RemoveNull();
        foreach (string otherName in otherNameArray)
        {
data.Add(otherName.Trim());
indetie.Add(dataTemp.Trim());

unit.Add(sheet.RCellValue<string>(i,5).Trim());//Units
channel.Add( sheet.RCellValue<int>(i,7));//Convert.ToInt32(row["输出频道编号"])
description.Add( sheet.RCellValue<string>(i,3).Trim());//"Description"
convention.Add(sheet.RCellValue<string>(i,4).Trim());//"Convention

string str1 = sheet.RCellValue<string>(i,8).Trim() as string ?? string.Empty;//"截面编号"
string str2 = sheet.RCellValue<string>(i,9).Trim() as string ?? string.Empty;//"数量编号"
string str3 = sheet.RCellValue<string>(i,10).Trim() as string ?? string.Empty;//"其他编号"
string strres = string.Empty;
string strres2 = string.Empty;

if (!string.IsNullOrEmpty(str1) && string.IsNullOrEmpty(str2) && string.IsNullOrEmpty(str3))
{
    strres = $"[{str1}]";
    strres2 = "[]";
}
else if (string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && string.IsNullOrEmpty(str3))
{
    strres = $"[{str2}]";
    strres2 = "[]";
}
else if (string.IsNullOrEmpty(str1) && string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
{
    strres = $"[{str3}]";
    strres2 = "[]";
}
else if (!string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && string.IsNullOrEmpty(str3))
{
    strres = $"[{str1},{str2}]";
    strres2 = "[,]";
}
else if (!string.IsNullOrEmpty(str1) && string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
{
    strres = $"[{str1},{str3}]";
    strres2 = "[,]";
}
else if (string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
{
    strres = $"[{str2},{str3}]";
    strres2 = "[,]";
}
else if (!string.IsNullOrEmpty(str1) && !string.IsNullOrEmpty(str2) && !string.IsNullOrEmpty(str3))
{
    strres = $"[{str1},{str2},{str3}]";
    strres2 = "[,,]";
}


index.Add(strres);
ipx.Add(strres2);
        }
    }
}
return (fredata,data,indetie,unit,channel,description,convention,index,ipx,Category,Titlechannel,TitlechannelDisp);

}


### 1.2 MBD模块

In [38]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\HawtC2\modules\MBD\MBD_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("MBD");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel)=getsheetdata("MBD",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 MBD 多体动力学的有效输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"MBDL");


file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.MBD");
file.WriteLine("{");
writesummary(file,"MBD输出参数类");
file.WriteLine("    public class MBD_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> MBD_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> MBD_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> MBD_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[K]")
    {
        file.Write("1","");
    }
    if(index[i]=="[J]")
    {
        file.Write("2","");
    }
    if(index[i]=="[J,K]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double MBD_GetParamOutput(string param, int J, MBD_AllOuts AllOuts,int K=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MBD_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct MBD_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");

//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum MBD_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");

file.WriteLine("    }");
file.WriteLine("}");

file.Outfinish(); 

### 1.3 MoptL 模块

### 1.4 APIL_hawtc.APi

In [ ]:

LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\OpenhastV7\modules\APIL\\ApiL_OutputParam.cs";


var sheet=excle.GetSheet("MoptKeyType");

//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 获取当前表格有效行数
int row=excle.RowCount(sheet);


file.WriteLine("");
writeinf(file,"APIL");
file.WriteLine("");
file.WriteLine("using OpenWECD.IO.IO;","using OpenWECD.IO.Log;","using System;","using System.Collections.Frozen;"
,"using System.Collections.Generic;","using System.Linq;","using System.Reflection;");
file.WriteLine("");
file.WriteLine("namespace OpenWECD.APIL");
file.WriteLine("{");
file.WriteLine("    /// <summary>","    /// API函数当中的参数表","    /// </summary>");
file.WriteLine("    public class ApiL_OutputParam");
file.WriteLine("    {");
file.WriteLine("        /// <summary>","        /// 构造函数,初始化API当中的参数表","        /// </summary>");
file.WriteLine("        public ApiL_OutputParam()");
file.WriteLine("        {");
file.WriteLine("//#对ApiYamlNameAndTypeDictionary进行初始化");
file.WriteLine("var ApiYamlNameAndTypeDictionarytemp = new Dictionary<string, Type>();");
file.WriteLine("foreach (var item in ApiYamlNameAndTypeNameDictionary)");
file.WriteLine("{");
file.WriteLine("    Type res = Type.GetType(item.Value);");
file.WriteLine("    if (res == null)");
file.WriteLine("    {");
file.WriteLine("        if (item.Value.Contains(\"OpenWECD\"))//是枚举，单独来看");
file.WriteLine("        {");
file.WriteLine("Assembly assembly = Assembly.Load(Otherhelper.GetCurrentProjectName());");
file.WriteLine("res = assembly.GetType(item.Value);");
file.WriteLine("        }");
file.WriteLine("        if (res == null)");
file.WriteLine("        {");
file.WriteLine("LogHelper.WarnLog(item.Key+ \"INI ERROR \",FunctionName: \"ApiL_OutputParam\");");
file.WriteLine("        }");
file.WriteLine("    }");

file.WriteLine("    //if(!TypeList.Contains(res))");
file.WriteLine("    //{");
file.WriteLine("    //    TypeList.Add(res);");
file.WriteLine("    //}");
file.WriteLine("");
file.WriteLine("    ApiYamlNameAndTypeDictionarytemp.Add(item.Key, res);");
file.WriteLine("}");
file.WriteLine("ApiYamlNameAndTypeDictionary = ApiYamlNameAndTypeDictionarytemp.ToFrozenDictionary();");
file.WriteLine("TypeList = ApiYamlNameAndTypeDictionary.Values.Distinct().ToArray();");
file.WriteLine("        }");
file.WriteLine("");

file.WriteLine("        /// <summary>","        /// 输出参数的参数表,用来保存yaml变量和类型和名称信息","        /// </summary>");
file.WriteLine("        public static FrozenDictionary<string, string> ApiYamlNameAndTypeNameDictionary = new Dictionary<string, string>()");
file.WriteLine("        {");
for(int i=1;i<row;i++)
{
    file.WriteLine("{\""+sheet.RCellValue<string>(i,0)+"\",\""+sheet.RCellValue<string>(i,1)+"\"},");
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的参数表,pst APi 当中变量名称及其对应模块的名称
file.WriteLine("        /// <summary>","        /// pst APi 当中变量名称及其对应模块的名称","        /// </summary>");
file.WriteLine("        public static FrozenDictionary<string, string> ApiOutNameAndMoudleNameDirtionary = new Dictionary<string, string>()");
file.WriteLine("        {");
//# 首先处理AeroL
for(int i=0;i<OpenWECD.AeroL.AeroL_OutputParam.AER_OutParChannel.Keys.Count();i++)
{
    file.WriteLine("{\""+OpenWECD.AeroL.AeroL_OutputParam.AER_OutParUnit.Keys[i]+"\",\""+"AeroL"+"\"},");
}


//# 再处理MBDL
for(int i=0;i<OpenWECD.MBD.MBD_OutputParam.MBD_OutParChannel.Keys.Count();i++)
{
    file.WriteLine("{\""+OpenWECD.MBD.MBD_OutputParam.MBD_OutParChannel.Keys[i]+"\",\""+"MBDL"+"\"},");
}


//# 再处理WINDL
for(int i=0;i<OpenWECD.WindL.WindL_OutputParam.WND_OutParChannel.Keys.Count();i++)
{
    file.WriteLine("{\""+OpenWECD.WindL.WindL_OutputParam.WND_OutParUnit.Keys.ToArray()[i]+"\",\""+"WindL"+"\"},");
}

//# 再处理ControL
for(int i=0;i<OpenWECD.ControlL.ControL_OutputParam.CON_OutParChannel.Keys.Count();i++)
{
    file.WriteLine("{\""+OpenWECD.ControlL.ControL_OutputParam.CON_OutParChannel.Keys.ToArray()[i]+"\",\""+"ControL"+"\"},");
}



file.WriteLine("        }.ToFrozenDictionary();");

file.WriteLine("");
file.WriteLine("        /// <summary>","        /// 输出参数的参数表,用来保存yam变量和类型信息","        /// </summary>");
file.WriteLine("        public  FrozenDictionary<string, Type> ApiYamlNameAndTypeDictionary;");
file.WriteLine("        /// <summary>","        /// 保存了全部的类型种类信息，没有重复！","        /// </summary>");
file.WriteLine("        public Type[] TypeList;");
file.WriteLine("    }");
file.WriteLine("}");
sheet.RCellValue<string>(0,1);

file.Outfinish();
typeof(OpenWECD.AeroL.AeroL_Loadchannels)


### 1.5 AeroL 模块

In [4]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\HawtC2\modules\AeroL\AeroL_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("AeroL");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel)=getsheetdata("AeroL",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 AeroL 非定常与定常空气动力学模块的有效输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"AeroL_OutputParam");


file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.AeroL");
file.WriteLine("{");
writesummary(file,"MBD输出参数类");
file.WriteLine("    public class AeroL_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> AER_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> AER_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> AER_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[K]")
    {
        file.Write("1","");
    }
    if(index[i]=="[J]")
    {
        file.Write("2","");
    }
    if(index[i]=="[J,K]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double AER_GetParamOutput(string param, int J, AER_AllOuts AllOuts,int K=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MBD_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct AER_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");


//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum AER_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");

file.WriteLine("    }");
file.WriteLine("}");


file.WriteLine("    }");
file.WriteLine("}");

file.Outfinish(); 

### 1.6 BeamL 模块

In [ ]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\OpenhastV7\modules\BeamL/BeamL_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("BeamL");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel)=getsheetdata("BeamL",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 BeamL 非线性与线性各向异性梁模块的有效输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"BeamL_OutputParam");


file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.BeamL");
file.WriteLine("{");
writesummary(file,"MBD输出参数类");
file.WriteLine("    public class BeamL_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> BEA_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> BEA_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> BEA_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[K]")
    {
        file.Write("1","");
    }
    if(index[i]=="[J]")
    {
        file.Write("2","");
    }
    if(index[i]=="[J,K]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double BEA_GetParamOutput(string param, int J, BEA_AllOuts AllOuts,int K=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MBD_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct BEA_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");


//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum BEA_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");

file.WriteLine("    }");
file.WriteLine("}");




file.WriteLine("    }");
file.WriteLine("}");

file.Outfinish(); 

### 1.7 ControL 模块

In [ ]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\OpenhastV7\modules\ControlL\ControL_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("ControL");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel)=getsheetdata("ControL",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 ControL 控制器模块的有效输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"ControL_OutputParam");


file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.ControlL");
file.WriteLine("{");
writesummary(file,"MBD输出参数类");
file.WriteLine("    public class ControL_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> CON_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> CON_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> CON_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[K]")
    {
        file.Write("1","");
    }
    if(index[i]=="[J]")
    {
        file.Write("2","");
    }
    if(index[i]=="[J,K]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double CON_GetParamOutput(string param, int J, CON_AllOuts AllOuts,int K=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MBD_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct CON_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");


//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum BEA_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");

file.WriteLine("    }");
file.WriteLine("}");




file.WriteLine("    }");
file.WriteLine("}");

file.Outfinish(); 

### 1.8 FEML 模块

In [ ]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\OpenhastV7\modules\FEML\FEM_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("FEML");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel)=getsheetdata("FEML",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 FEML 有限元模块的有效输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"FEM_OutputParam");


file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.FEML");
file.WriteLine("{");
writesummary(file,"MBD输出参数类");
file.WriteLine("    public class FEM_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> FEM_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> FEM_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> FEM_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[K]")
    {
        file.Write("1","");
    }
    if(index[i]=="[J]")
    {
        file.Write("2","");
    }
    if(index[i]=="[J,K]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double FEM_GetParamOutput(string param, int J, FEM_AllOuts AllOuts,int K=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MBD_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct FEM_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");

//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum FEM_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");

file.WriteLine("    }");
file.WriteLine("}");





file.WriteLine("    }");
file.WriteLine("}");

file.Outfinish(); 

### 1.9 WindL 模块

In [ ]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\HawtC2\modules\WindL\WindL_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("WindL");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel)=getsheetdata("WindL",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 WindL 外部风环境模块的有效输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"WindL_OutputParam");


file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.WindL");
file.WriteLine("{");
writesummary(file,"MBD输出参数类");
file.WriteLine("    public class WindL_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> WND_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> WND_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> WND_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[Z]")
    {
        file.Write("1","");
    }
    if(index[i]=="[Y]")
    {
        file.Write("2","");
    }
    if(index[i]=="[Y,Z]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double WND_GetParamOutput(string param, int Y, WND_AllOuts AllOuts,int Z=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MBD_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct WND_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");

//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum WND_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");

file.WriteLine("    }");
file.WriteLine("}");

file.Outfinish(); 

### 1.10 HydroL 模块

In [10]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\HawtC2\modules\HydroL\HydroL_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("HydroL");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel)=getsheetdata("HydroL",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 HydroL 水动力输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"HydroL_OutputParam");

file.WriteLine();
file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;","using SourceGeneration.Reflection;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.HydroL");
file.WriteLine("{");
writesummary(file,"MBD输出参数类");
file.WriteLine("    public class HydroL_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> HYD_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> HYD_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> HYD_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
   if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[K]")
    {
        file.Write("1","");
    }
    if(index[i]=="[J]")
    {
        file.Write("2","");
    }
    if(index[i]=="[J,K]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double HYD_GetParamOutput(string param, int Y, HYD_AllOuts AllOuts,int Z=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MBD_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct HYD_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");

//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum HYD_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");


file.WriteLine("    }");
file.WriteLine("}");
file.Outfinish(); 


### 1.11 MSA 模块

In [4]:
LogHelper.DisplayInformation();
//# 设置代码路径
string ApiL_OutputParamFilePath=@"G:\2026\HawtC2\modules\MSAL\MSA_OutputParam.cs";

//# 获取对应表格数据
var sheet=excle.GetSheet("MSA");
//# 初始化文件读写生成器
var file=new OpenWECD.IO.IO.OutFile(ApiL_OutputParamFilePath);

//# 
(var adata,var data,var namep,var unit,var channel,var description,var convention,var index,var ipx,var Category,var TitleChannel,var TitlechannelDisp)=getsheetdata("MSA",excle);
//# 输出信息
file.WriteLine($"//! 自动生成 MSA 模型线性化与稳定求解器的有效输出变量数量为:{data.Count}");
//# 写入文件头
writeinf(file,"MSA");


file.WriteLine("using System.Collections.Generic;","using System.Collections.Frozen;","using OpenWECD.IO.Log;","using MathNet.Numerics.LinearAlgebra;",
   "using OpenWECD.IO.Interface1;");

//#写入命名空间
file.WriteLine("namespace OpenWECD.MSAL");
file.WriteLine("{");
writesummary(file,"MSA输出参数类");
file.WriteLine("    public class MSA_OutputParam");
file.WriteLine("    {");


//! 写入输出参数的单位参数表
writesummary(file,"输出参数的参数表,用来保存变量和单位信息");
file.WriteLine("        public static FrozenDictionary<string, string> MSA_OutParUnit = new Dictionary<string, string>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    file.Write("\"","");
    writeo(file, unit[i], 10);
    file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

//# 输出参数的通道表,用来保存变量和通道信息
writesummary(file,"输出参数的通道表,用来保存变量和通道信息");
file.WriteLine("        public static FrozenDictionary<string, int> MSA_OutParChannel = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    writeo(file, channel[i].ToString(), 10);
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");


//# 输出参数的描述表,用来保存变量和描述信息
writesummary(file,"输出参数的维度 0 表示没有维度 1 表示只要部件编号 2 表示只要部件截面 3 表示部件编号和截面编号都要");
file.WriteLine("        public static FrozenDictionary<string, int> MSA_OutParDim = new Dictionary<string, int>()");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("            ","");
    file.Write("{","");
    file.Write("\"","");
    writeo(file, data[i], 10);
    file.Write("\"","");
    file.Write(",","");
    //file.Write("\"","");
    if(index[i]==string.Empty)
    {
        file.Write("0","");
    }
    if(index[i]=="[K]")
    {
        file.Write("1","");
    }
    if(index[i]=="[J]")
    {
        file.Write("2","");
    }
    if(index[i]=="[J,K]")
    {
        file.Write("3","");
    }
    //file.Write("\"","");
    file.Write("}","");
    file.Write(",","");

    if (i % 4 == 0)
    {
        file.Write("\n","");
    }
}
file.WriteLine("        }.ToFrozenDictionary();");

writesummary(file,"依据变量名称,输出变量值");
file.WriteLine("        public static double MSA_GetParamOutput(string param, int J, MSA_AllOuts AllOuts,int K=0)");
file.WriteLine("        {");
file.WriteLine("            switch (param)");
file.WriteLine("            {");
for (int i = 0; i < unit.Count; i++)
{
    file.Write("                case ","");
    file.Write("\"","");
    writeo(file,data[i],10);
    file.Write("\"","");
    file.WriteLine(":");
    file.Write("                    return AllOuts.","");
    file.Write(namep[i],"");
    file.Write(index[i],"");
    file.WriteLine(";");


}
    file.WriteLine("");
    file.WriteLine("                default:");
    file.WriteLine("                    LogHelper.ErrorLog($\"{param} not Support!,Please Visit OutputParList or www.openwecd.fun \",FunctionName: \"MSA_GetParamOutput\");");
    file.WriteLine("                    return 0;");
file.WriteLine("            }");
file.WriteLine("        }");



//# 输出变量结构体
writesummary(file,"阐述变量的结构体");
file.WriteLine("        public struct MSA_AllOutss");
file.WriteLine("        {");
for (int i = 0; i < unit.Count; i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {description[i]}  {convention[i]}"+$" unit:{unit[i]}");
    file.WriteLine("            /// </summary>");
        file.Write("            public double ","");
        file.Write(ipx[i]+" ","");
        file.Write(data[i],"");
        file.WriteLine(";");
}
file.WriteLine("        }");



//# 输出初始化变量



//# 输出频道枚举
writesummary(file,"载荷输出的通道枚举体,要和c#自动化文件对应");
file.WriteLine("        [SourceReflection]");
file.WriteLine("        public enum MSA_Loadchannels");
file.WriteLine("        {");
for(int i=0;i<Category.Count;i++)
{
    file.WriteLine("            /// <summary>");
    file.WriteLine($"            /// {TitlechannelDisp[i]}");
    file.WriteLine("            /// </summary>");
    file.WriteLine($"            {Category[i]} = {TitleChannel[i]},");
}
file.WriteLine("        }");

file.WriteLine("    }");
file.WriteLine("}");

file.Outfinish(); 

OpenWECD.Microsoft.DotNet.Interactive.App - Tel:13935201274  E:1821216780@qq.com http://www.hawtc.cn/
************************************************************************************
Microsoft.DotNet.Interactive.App Vision:VVSCODE CALL_Release_x64,BuildAt: TESTMODE MathACC:SIMD_by赵子祯
 > Publish: The Microsoft.DotNet.Interactive.App powered by Tg Team in China,and written by zzz@Copyrigth 
 > Licensd: 超级用户 - Copyright (c)ZHAO ZIZHEN Lic at 2025-02-25 


In [5]:
TitlechannelDisp